# 🔗 Configuration des API Endpoints

**Module :** 00-GenAI-Environment  
**Niveau :** 🟢 Débutant  
**Technologies :** OpenRouter, OpenAI API, Monitoring  
**Durée estimée :** 20 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Configurer et tester les connexions API
- [ ] Valider les clés API et permissions
- [ ] Effectuer des benchmarks de performance
- [ ] Diagnostiquer les problèmes courants
- [ ] Monitorer l'utilisation et les coûts
- [ ] Optimiser les paramètres de connexion

## 📚 APIs Configurées

### 🚀 OpenRouter API
- **DALL-E 3** : `openai/dall-e-3`
- **GPT-5** : `openai/gpt-5` 
- **Qwen-Image-Edit-2509** : `qwen/qwen-image-edit-2509`
- **FLUX-1** : Modèles de génération avancée

### 🤖 OpenAI Direct API
- **DALL-E 3** : Accès direct OpenAI
- **GPT-4 Vision** : Analyse d'images
- **Whisper** : Transcription audio

## ⚙️ Prérequis

- Fichier `.env` configuré avec les clés API
- Accès internet stable
- Python 3.11+ avec packages requis

In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

# Configuration des tests
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"               

# Tests à effectuer
test_openrouter = True             # Tester OpenRouter API
test_openai_direct = True          # Tester OpenAI API directe
run_benchmarks = True              # Benchmarks de performance
check_rate_limits = True           # Vérifier les limites de taux
monitor_costs = True               # Monitoring des coûts

# Configuration benchmarks
benchmark_iterations = 3           # Nombre de tests par endpoint
benchmark_timeout = 30             # Timeout par test (secondes)
model_list_limit = 10              # Limite modèles à tester

# Paramètres monitoring
export_results = True              # Exporter résultats JSON
generate_report = True             # Générer rapport HTML
show_detailed_logs = False         # Logs détaillés (verbose)

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import requests
import time
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
import logging
from urllib.parse import urljoin
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging, load_genai_config
        print("✅ Helpers GenAI importés")
    except ImportError:
        print("⚠️  Helpers GenAI non disponibles - mode autonome")

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('api_configuration')

# Chargement des variables d'environnement
env_file = GENAI_ROOT / '.env'
if env_file.exists():
    load_dotenv(env_file)
    print(f"✅ Variables d'environnement chargées depuis {env_file}")
else:
    print(f"⚠️  Fichier .env non trouvé: {env_file}")
    print("💡 Utilisez .env.template comme base")

print(f"\n🔗 Configuration des API Endpoints")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Mode: {notebook_mode}, Benchmarks: {run_benchmarks}")
print(f"🏷️  Tests: OpenRouter={test_openrouter}, OpenAI={test_openai_direct}")

In [ ]:
# Configuration et validation des clés API
print("\n🔑 VALIDATION DES CLÉS API")
print("=" * 35)

# Dictionnaire de configuration des APIs
api_config = {
    "openrouter": {
        "name": "OpenRouter",
        "base_url": "https://openrouter.ai/api/v1",
        "key_env": "OPENROUTER_API_KEY",
        "key_value": None,
        "headers": {},
        "endpoints": {
            "models": "/models",
            "chat": "/chat/completions",
            "images": "/images/generations"
        },
        "test_model": "openai/gpt-4o-mini",
        "status": "unknown"
    },
    
    "openai": {
        "name": "OpenAI Direct",
        "base_url": "https://api.openai.com/v1",
        "key_env": "OPENAI_API_KEY",
        "key_value": None,
        "headers": {},
        "endpoints": {
            "models": "/models",
            "chat": "/chat/completions",
            "images": "/images/generations"
        },
        "test_model": "gpt-4o-mini",
        "status": "unknown"
    }
}

# Validation et configuration des clés
for api_name, config in api_config.items():
    print(f"\n🔍 {config['name']}")
    print("-" * 25)
    
    # Récupération de la clé API
    api_key = os.getenv(config['key_env'])
    
    if api_key:
        config['key_value'] = api_key
        
        # Configuration headers
        if api_name == "openrouter":
            config['headers'] = {
                "Authorization": f"Bearer {api_key}",
                "HTTP-Referer": "https://coursia.myia.io",
                "X-Title": "CoursIA GenAI Configuration",
                "Content-Type": "application/json"
            }
        else:  # OpenAI
            config['headers'] = {
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            }
        
        # Masquage partiel de la clé pour l'affichage
        masked_key = f"{api_key[:8]}...{api_key[-4:]}" if len(api_key) > 12 else "***"
        print(f"✅ Clé API: {masked_key}")
        print(f"📍 URL base: {config['base_url']}")
        print(f"🎯 Modèle test: {config['test_model']}")
        config['status'] = "configured"
    else:
        print(f"❌ Variable manquante: {config['key_env']}")
        print(f"💡 Ajoutez {config['key_env']}=your_key_here dans .env")
        config['status'] = "missing"

# Résumé de la configuration
print(f"\n📊 RÉSUMÉ CONFIGURATION")
print("=" * 30)
configured_apis = sum(1 for c in api_config.values() if c['status'] == 'configured')
total_apis = len(api_config)
print(f"🎯 APIs configurées: {configured_apis}/{total_apis}")

if configured_apis == 0:
    print("⚠️  Aucune API configurée - vérifiez votre fichier .env")
    print("📝 Utilisez .env.template comme référence")
elif configured_apis < total_apis:
    print("⚠️  Configuration partielle - certains tests seront ignorés")
else:
    print("✅ Toutes les APIs sont configurées - tests complets possibles")

In [ ]:
# Tests de connectivité des APIs
print("\n🌐 TESTS DE CONNECTIVITÉ")
print("=" * 32)

# Fonction de test de connectivité
def test_api_connectivity(api_name: str, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Teste la connectivité d'une API.
    
    Args:
        api_name: Nom de l'API
        config: Configuration de l'API
        
    Returns:
        Dict avec résultats du test
    """
    result = {
        "api": api_name,
        "name": config['name'],
        "success": False,
        "response_time": None,
        "status_code": None,
        "models_count": 0,
        "error": None,
        "timestamp": datetime.now().isoformat()
    }
    
    if config['status'] != 'configured':
        result['error'] = "API non configurée"
        return result
    
    try:
        print(f"\n🔍 Test {config['name']}...")
        
        # Test endpoint /models
        start_time = time.time()
        
        response = requests.get(
            urljoin(config['base_url'], config['endpoints']['models']),
            headers=config['headers'],
            timeout=benchmark_timeout
        )
        
        end_time = time.time()
        result['response_time'] = round(end_time - start_time, 3)
        result['status_code'] = response.status_code
        
        if response.status_code == 200:
            data = response.json()
            
            if api_name == "openrouter":
                models = data.get('data', [])
                result['models_count'] = len(models)
                
                # Filtrage des modèles GenAI Images
                genai_models = [
                    m for m in models 
                    if any(keyword in m.get('id', '').lower() 
                          for keyword in ['dall-e', 'gpt-5', 'gpt-4', 'flux', 'qwen'])
                ]
                
                print(f"✅ Connexion réussie - {result['response_time']}s")
                print(f"📊 Modèles totaux: {result['models_count']}")
                print(f"🎨 Modèles GenAI Images: {len(genai_models)}")
                
                # Affichage des modèles GenAI principaux
                if genai_models:
                    print(f"\n🎯 Modèles prioritaires détectés:")
                    for model in genai_models[:5]:
                        context = model.get('context_length', 'N/A')
                        pricing = model.get('pricing', {})
                        prompt_price = pricing.get('prompt', 'N/A')
                        print(f"  • {model['id']} - Contexte: {context}, Prix: ${prompt_price}")
            
            else:  # OpenAI
                models = data.get('data', [])
                result['models_count'] = len(models)
                
                print(f"✅ Connexion réussie - {result['response_time']}s")
                print(f"📊 Modèles disponibles: {result['models_count']}")
                
                # Affichage des modèles principaux
                key_models = [m for m in models if m.get('id') in ['gpt-4o', 'gpt-4o-mini', 'dall-e-3']]
                if key_models:
                    print(f"\n🎯 Modèles clés détectés:")
                    for model in key_models:
                        print(f"  • {model['id']} - Propriétaire: {model.get('owned_by', 'N/A')}")
            
            result['success'] = True
            
        else:
            error_msg = f"HTTP {response.status_code}"
            try:
                error_data = response.json()
                error_msg += f": {error_data.get('error', {}).get('message', 'Erreur inconnue')}"
            except:
                pass
            
            result['error'] = error_msg
            print(f"❌ Échec: {error_msg}")
            
    except requests.exceptions.Timeout:
        result['error'] = f"Timeout après {benchmark_timeout}s"
        print(f"⏱️  Timeout: {result['error']}")
    except requests.exceptions.ConnectionError:
        result['error'] = "Erreur de connexion réseau"
        print(f"🌐 Réseau: {result['error']}")
    except Exception as e:
        result['error'] = str(e)[:100]
        print(f"❌ Erreur: {result['error']}")
    
    return result

# Exécution des tests de connectivité
connectivity_results = []

for api_name, config in api_config.items():
    if (api_name == "openrouter" and test_openrouter) or \
       (api_name == "openai" and test_openai_direct):
        result = test_api_connectivity(api_name, config)
        connectivity_results.append(result)
    else:
        print(f"\n⏭️  {config['name']} - Test désactivé")

# Résumé des tests de connectivité
print(f"\n📈 RÉSUMÉ CONNECTIVITÉ")
print("=" * 28)
successful_tests = sum(1 for r in connectivity_results if r['success'])
total_tests = len(connectivity_results)

print(f"🎯 Tests réussis: {successful_tests}/{total_tests}")

if successful_tests > 0:
    avg_response_time = sum(r['response_time'] for r in connectivity_results if r['success']) / successful_tests
    total_models = sum(r['models_count'] for r in connectivity_results if r['success'])
    
    print(f"⚡ Temps moyen: {avg_response_time:.3f}s")
    print(f"🔢 Modèles totaux: {total_models}")
else:
    print("❌ Aucune connexion réussie")

In [ ]:
# Benchmarks de performance des APIs
if run_benchmarks and connectivity_results:
    print("\n⚡ BENCHMARKS DE PERFORMANCE")
    print("=" * 35)
    
    # Fonction de benchmark
    def benchmark_api_endpoint(api_name: str, config: Dict[str, Any], 
                              endpoint: str, iterations: int = 3) -> Dict[str, Any]:
        """
        Benchmark d'un endpoint API.
        
        Args:
            api_name: Nom de l'API
            config: Configuration de l'API
            endpoint: Endpoint à tester
            iterations: Nombre d'itérations
            
        Returns:
            Dict avec métriques de performance
        """
        
        if config['status'] != 'configured':
            return {"error": "API non configurée"}
        
        response_times = []
        errors = 0
        
        print(f"\n🏃 Benchmark {config['name']} - {endpoint}")
        print(f"🔄 {iterations} itérations...")
        
        for i in range(iterations):
            try:
                start_time = time.time()
                
                response = requests.get(
                    urljoin(config['base_url'], config['endpoints'][endpoint]),
                    headers=config['headers'],
                    timeout=benchmark_timeout
                )
                
                end_time = time.time()
                response_time = end_time - start_time
                
                if response.status_code == 200:
                    response_times.append(response_time)
                    print(f"  ✅ Itération {i+1}: {response_time:.3f}s")
                else:
                    errors += 1
                    print(f"  ❌ Itération {i+1}: HTTP {response.status_code}")
                
                # Délai entre les requêtes pour éviter le rate limiting
                if i < iterations - 1:
                    time.sleep(0.5)
                    
            except Exception as e:
                errors += 1
                print(f"  ❌ Itération {i+1}: {str(e)[:50]}...")
        
        if response_times:
            return {
                "api": api_name,
                "endpoint": endpoint,
                "success_rate": len(response_times) / iterations,
                "avg_response_time": sum(response_times) / len(response_times),
                "min_response_time": min(response_times),
                "max_response_time": max(response_times),
                "total_requests": iterations,
                "successful_requests": len(response_times),
                "errors": errors,
                "timestamp": datetime.now().isoformat()
            }
        else:
            return {
                "api": api_name,
                "endpoint": endpoint,
                "error": "Aucune requête réussie",
                "errors": errors,
                "total_requests": iterations
            }
    
    # Exécution des benchmarks
    benchmark_results = []
    
    for result in connectivity_results:
        if result['success']:
            api_name = result['api']
            config = api_config[api_name]
            
            # Test de l'endpoint /models (le plus stable)
            bench_result = benchmark_api_endpoint(
                api_name, config, "models", benchmark_iterations
            )
            benchmark_results.append(bench_result)
    
    # Analyse des résultats de benchmark
    if benchmark_results:
        print(f"\n📊 ANALYSE DES BENCHMARKS")
        print("=" * 30)
        
        # Tableau comparatif
        print(f"\n{'API':<15} {'Succès':<8} {'Temps Moy':<12} {'Min':<8} {'Max':<8} {'Erreurs':<8}")
        print("-" * 65)
        
        for bench in benchmark_results:
            if 'error' not in bench:
                api = bench['api'].title()
                success_pct = f"{bench['success_rate']*100:.1f}%"
                avg_time = f"{bench['avg_response_time']:.3f}s"
                min_time = f"{bench['min_response_time']:.3f}s"
                max_time = f"{bench['max_response_time']:.3f}s"
                errors = str(bench['errors'])
                
                print(f"{api:<15} {success_pct:<8} {avg_time:<12} {min_time:<8} {max_time:<8} {errors:<8}")
        
        # Recommandations de performance
        print(f"\n💡 RECOMMANDATIONS")
        print("=" * 22)
        
        best_apis = [b for b in benchmark_results if 'error' not in b and b['success_rate'] > 0.8]
        
        if best_apis:
            fastest_api = min(best_apis, key=lambda x: x['avg_response_time'])
            most_reliable = max(best_apis, key=lambda x: x['success_rate'])
            
            print(f"🚀 API la plus rapide: {fastest_api['api'].title()} ({fastest_api['avg_response_time']:.3f}s)")
            print(f"🛡️  API la plus fiable: {most_reliable['api'].title()} ({most_reliable['success_rate']*100:.1f}% succès)")
            
            # Conseils d'optimisation
            slow_apis = [b for b in best_apis if b['avg_response_time'] > 2.0]
            if slow_apis:
                print(f"\n⚠️  APIs lentes détectées:")
                for api in slow_apis:
                    print(f"   • {api['api'].title()}: {api['avg_response_time']:.3f}s")
                print(f"💡 Considérez un timeout adapté et des retries")
        else:
            print(f"❌ Aucune API avec performance acceptable")
            print(f"🔧 Vérifiez votre connexion réseau et les clés API")
else:
    if not run_benchmarks:
        print(f"\n⏭️  Benchmarks désactivés (run_benchmarks = False)")
    else:
        print(f"\n⚠️  Aucune connexion réussie pour les benchmarks")

In [ ]:
# Vérification des limites de taux (Rate limits)
if check_rate_limits and connectivity_results:
    print("\n🚦 VÉRIFICATION DES RATE LIMITS")
    print("=" * 40)
    
    def check_rate_limits_api(api_name: str, config: Dict[str, Any]) -> Dict[str, Any]:
        """
        Vérifie les limites de taux d'une API.
        """
        
        if config['status'] != 'configured':
            return {"error": "API non configurée"}
        
        rate_info = {
            "api": api_name,
            "limits_detected": False,
            "headers_info": {},
            "request_count": 0,
            "rate_limited": False,
            "reset_time": None
        }
        
        print(f"\n🔍 Test rate limits {config['name']}")
        
        try:
            # Test avec plusieurs requêtes rapprochées
            for i in range(5):
                response = requests.get(
                    urljoin(config['base_url'], config['endpoints']['models']),
                    headers=config['headers'],
                    timeout=10
                )
                
                rate_info['request_count'] += 1
                
                # Analyse des headers de rate limiting
                headers_to_check = [
                    'x-ratelimit-limit',
                    'x-ratelimit-remaining', 
                    'x-ratelimit-reset',
                    'retry-after',
                    'x-ratelimit-limit-requests',
                    'x-ratelimit-remaining-requests'
                ]
                
                found_headers = {}
                for header in headers_to_check:
                    value = response.headers.get(header)
                    if value:
                        found_headers[header] = value
                        rate_info['limits_detected'] = True
                
                if found_headers:
                    rate_info['headers_info'].update(found_headers)
                
                if response.status_code == 429:
                    rate_info['rate_limited'] = True
                    retry_after = response.headers.get('retry-after')
                    if retry_after:
                        rate_info['reset_time'] = int(retry_after)
                    print(f"⚠️  Rate limit atteint après {i+1} requêtes")
                    break
                elif response.status_code != 200:
                    print(f"❌ Erreur HTTP {response.status_code} à la requête {i+1}")
                    break
                
                # Petit délai entre requêtes
                time.sleep(0.1)
            
            # Affichage des résultats
            if rate_info['limits_detected']:
                print(f"✅ Headers de rate limiting détectés")
                for header, value in rate_info['headers_info'].items():
                    print(f"   {header}: {value}")
            else:
                print(f"ℹ️  Aucun header de rate limiting détecté")
            
            if rate_info['rate_limited']:
                reset_info = f" (reset dans {rate_info['reset_time']}s)" if rate_info['reset_time'] else ""
                print(f"🚨 Rate limiting actif{reset_info}")
            else:
                print(f"✅ Aucun rate limiting rencontré")
        
        except Exception as e:
            rate_info['error'] = str(e)[:100]
            print(f"❌ Erreur test rate limits: {rate_info['error']}")
        
        return rate_info
    
    # Test des rate limits pour chaque API
    rate_limit_results = []
    
    for result in connectivity_results:
        if result['success']:
            api_name = result['api']
            config = api_config[api_name]
            
            rate_result = check_rate_limits_api(api_name, config)
            rate_limit_results.append(rate_result)
    
    # Résumé des rate limits
    if rate_limit_results:
        print(f"\n📋 RÉSUMÉ RATE LIMITS")
        print("=" * 25)
        
        for result in rate_limit_results:
            api = result['api'].title()
            detected = "Oui" if result.get('limits_detected') else "Non"
            limited = "Oui" if result.get('rate_limited') else "Non"
            
            print(f"🔧 {api}:")
            print(f"   Headers détectés: {detected}")
            print(f"   Rate limiting: {limited}")
            
            if result.get('headers_info'):
                limit = result['headers_info'].get('x-ratelimit-limit') or result['headers_info'].get('x-ratelimit-limit-requests')
                remaining = result['headers_info'].get('x-ratelimit-remaining') or result['headers_info'].get('x-ratelimit-remaining-requests')
                
                if limit and remaining:
                    usage = ((int(limit) - int(remaining)) / int(limit)) * 100
                    print(f"   Utilisation: {usage:.1f}% ({remaining}/{limit})")
        
        # Conseils pour gérer les rate limits
        print(f"\n💡 BONNES PRATIQUES RATE LIMITING")
        print("=" * 38)
        print(f"• Implementez des retries avec backoff exponentiel")
        print(f"• Respectez les headers Retry-After")
        print(f"• Utilisez des pools de connexions")
        print(f"• Monitorer l'utilisation en temps réel")
        print(f"• Avoir des fallbacks entre APIs")
else:
    if not check_rate_limits:
        print(f"\n⏭️  Vérification rate limits désactivée")
    else:
        print(f"\n⚠️  Aucune API disponible pour test rate limits")

In [ ]:
# Monitoring des coûts et utilisation
if monitor_costs and connectivity_results:
    print("\n💰 MONITORING DES COÛTS")
    print("=" * 30)
    
    # Estimation des coûts basée sur les modèles disponibles
    def estimate_usage_costs() -> Dict[str, Any]:
        """
        Estime les coûts d'utilisation des APIs.
        """
        
        cost_estimates = {
            "openrouter": {
                "dall-e-3": {
                    "standard": {"1024x1024": 0.04, "1792x1024": 0.08, "1024x1792": 0.08},
                    "hd": {"1024x1024": 0.08, "1792x1024": 0.12, "1024x1792": 0.12}
                },
                "gpt-5": {
                    "input_tokens_per_k": 0.005,  # Estimation
                    "output_tokens_per_k": 0.015
                },
                "gpt-4o": {
                    "input_tokens_per_k": 0.0025,
                    "output_tokens_per_k": 0.01
                }
            },
            "openai": {
                "dall-e-3": {
                    "standard": {"1024x1024": 0.04, "1792x1024": 0.08, "1024x1792": 0.08},
                    "hd": {"1024x1024": 0.08, "1792x1024": 0.12, "1024x1792": 0.12}
                },
                "gpt-4o": {
                    "input_tokens_per_k": 0.0025,
                    "output_tokens_per_k": 0.01
                }
            }
        }
        
        return cost_estimates
    
    cost_data = estimate_usage_costs()
    
    # Calculs d'exemple pour un usage typique CoursIA
    print("\n💡 ESTIMATION COÛTS USAGE COURSEIA")
    print("=" * 42)
    
    typical_usage = {
        "images_per_month": 100,  # Images générées par mois
        "chat_sessions_per_month": 50,  # Sessions d'analyse
        "avg_tokens_per_session": 2000  # Tokens moyens par session
    }
    
    print(f"📊 Usage typique mensuel estimé:")
    print(f"   🖼️  Images générées: {typical_usage['images_per_month']}")
    print(f"   💬 Sessions d'analyse: {typical_usage['chat_sessions_per_month']}")
    print(f"   🔤 Tokens par session: {typical_usage['avg_tokens_per_session']:,}")
    
    # Calcul des coûts par API
    monthly_costs = {}
    
    for api_name, api_costs in cost_data.items():
        api_total = 0
        
        print(f"\n💳 {api_name.title()} - Coûts mensuels estimés:")
        
        # DALL-E 3 costs
        if 'dall-e-3' in api_costs:
            dalle_cost = typical_usage['images_per_month'] * api_costs['dall-e-3']['standard']['1024x1024']
            api_total += dalle_cost
            print(f"   🎨 DALL-E 3 (100 images): ${dalle_cost:.2f}")
        
        # GPT costs (estimation basée sur input + output)
        gpt_models = [k for k in api_costs.keys() if 'gpt' in k]
        if gpt_models:
            model = gpt_models[0]  # Premier modèle GPT disponible
            if 'input_tokens_per_k' in api_costs[model]:
                total_tokens = typical_usage['chat_sessions_per_month'] * typical_usage['avg_tokens_per_session']
                # Estimation: 70% input, 30% output
                input_cost = (total_tokens * 0.7 / 1000) * api_costs[model]['input_tokens_per_k']
                output_cost = (total_tokens * 0.3 / 1000) * api_costs[model]['output_tokens_per_k']
                gpt_total = input_cost + output_cost
                api_total += gpt_total
                print(f"   🤖 {model.upper()} ({total_tokens:,} tokens): ${gpt_total:.2f}")
        
        monthly_costs[api_name] = api_total
        print(f"   💰 Total {api_name.title()}: ${api_total:.2f}/mois")
    
    # Comparaison et recommandations
    if monthly_costs:
        print(f"\n🏆 COMPARAISON ET RECOMMANDATIONS")
        print("=" * 40)
        
        cheapest_api = min(monthly_costs, key=monthly_costs.get)
        cheapest_cost = monthly_costs[cheapest_api]
        
        print(f"💡 API la plus économique: {cheapest_api.title()} (${cheapest_cost:.2f}/mois)")
        
        cost_diff = max(monthly_costs.values()) - min(monthly_costs.values())
        if cost_diff > 1:
            print(f"💸 Économie potentielle: ${cost_diff:.2f}/mois")
        
        # Tips d'optimisation des coûts
        print(f"\n🎯 CONSEILS D'OPTIMISATION")
        print("=" * 30)
        print(f"• Utilisez les modèles les plus adaptés au besoin")
        print(f"• Optimisez la longueur des prompts")
        print(f"• Implémentez du caching pour les réponses similaires")
        print(f"• Surveillez l'utilisation avec des alerts")
        print(f"• Testez différentes résolutions d'images selon l'usage")
else:
    if not monitor_costs:
        print(f"\n⏭️  Monitoring coûts désactivé")
    else:
        print(f"\n⚠️  Aucune API disponible pour monitoring coûts")

In [ ]:
# Guide de diagnostic et troubleshooting
print("\n🔧 GUIDE DE DIAGNOSTIC ET TROUBLESHOOTING")
print("=" * 55)

# Fonction de diagnostic automatique
def run_diagnostics() -> Dict[str, Any]:
    """
    Execute un diagnostic complet de la configuration.
    """
    
    diagnostics = {
        "timestamp": datetime.now().isoformat(),
        "env_file_status": "unknown",
        "api_keys_status": {},
        "connectivity_status": {},
        "common_issues": [],
        "recommendations": []
    }
    
    # Vérification fichier .env
    env_file = GENAI_ROOT / '.env'
    if env_file.exists():
        diagnostics['env_file_status'] = "found"
        try:
            with open(env_file, 'r') as f:
                env_content = f.read()
            if 'OPENROUTER_API_KEY' in env_content or 'OPENAI_API_KEY' in env_content:
                diagnostics['env_file_status'] = "configured"
        except:
            diagnostics['env_file_status'] = "read_error"
    else:
        diagnostics['env_file_status'] = "missing"
    
    # Status des clés API
    for api_name, config in api_config.items():
        key_status = "missing"
        if config.get('key_value'):
            key_status = "configured"
            # Test basique de format
            key = config['key_value']
            if len(key) < 20:
                key_status = "too_short"
            elif not key.replace('-', '').replace('_', '').isalnum():
                key_status = "invalid_format"
        
        diagnostics['api_keys_status'][api_name] = key_status
    
    # Status de connectivité
    for result in connectivity_results:
        api = result['api']
        status = "success" if result['success'] else "failed"
        error = result.get('error')
        
        diagnostics['connectivity_status'][api] = {
            "status": status,
            "error": error,
            "response_time": result.get('response_time')
        }
    
    # Détection des problèmes courants
    if diagnostics['env_file_status'] == "missing":
        diagnostics['common_issues'].append({
            "issue": "Fichier .env manquant",
            "solution": "Copiez .env.template vers .env et configurez vos clés API"
        })
    
    missing_keys = [api for api, status in diagnostics['api_keys_status'].items() if status == "missing"]
    if missing_keys:
        diagnostics['common_issues'].append({
            "issue": f"Clés API manquantes: {', '.join(missing_keys)}",
            "solution": "Ajoutez les clés manquantes dans le fichier .env"
        })
    
    failed_connections = [api for api, data in diagnostics['connectivity_status'].items() if data['status'] == "failed"]
    if failed_connections:
        diagnostics['common_issues'].append({
            "issue": f"Échecs de connexion: {', '.join(failed_connections)}",
            "solution": "Vérifiez votre connexion internet et les clés API"
        })
    
    # Recommandations générales
    if not diagnostics['common_issues']:
        diagnostics['recommendations'].append("✅ Configuration optimale détectée")
    else:
        diagnostics['recommendations'].append("🔧 Corrigez les problèmes identifiés ci-dessus")
    
    if any(data.get('response_time', 0) > 3 for data in diagnostics['connectivity_status'].values()):
        diagnostics['recommendations'].append("⚡ Connexion lente détectée - vérifiez votre réseau")
    
    return diagnostics

# Exécution du diagnostic
diag_results = run_diagnostics()

print(f"\n📋 RÉSULTATS DU DIAGNOSTIC")
print("=" * 32)

# Status fichier .env
env_status_icons = {
    "missing": "❌",
    "found": "⚠️",
    "configured": "✅",
    "read_error": "🔧"
}
env_icon = env_status_icons.get(diag_results['env_file_status'], "❓")
print(f"{env_icon} Fichier .env: {diag_results['env_file_status']}")

# Status clés API
print(f"\n🔑 Status des clés API:")
for api, status in diag_results['api_keys_status'].items():
    status_icon = "✅" if status == "configured" else "❌"
    print(f"   {status_icon} {api.title()}: {status}")

# Status connectivité
if diag_results['connectivity_status']:
    print(f"\n🌐 Status connectivité:")
    for api, data in diag_results['connectivity_status'].items():
        status_icon = "✅" if data['status'] == "success" else "❌"
        time_info = f" ({data['response_time']:.3f}s)" if data.get('response_time') else ""
        print(f"   {status_icon} {api.title()}: {data['status']}{time_info}")

# Problèmes détectés
if diag_results['common_issues']:
    print(f"\n🚨 PROBLÈMES DÉTECTÉS")
    print("=" * 25)
    for i, issue in enumerate(diag_results['common_issues'], 1):
        print(f"{i}. ❌ {issue['issue']}")
        print(f"   💡 Solution: {issue['solution']}")

# Recommandations
if diag_results['recommendations']:
    print(f"\n💡 RECOMMANDATIONS")
    print("=" * 20)
    for rec in diag_results['recommendations']:
        print(f"• {rec}")

print(f"\n📞 SUPPORT SUPPLÉMENTAIRE")
print("=" * 28)
print(f"• Documentation: docs/genai-troubleshooting-guide.md")
print(f"• Templates: docs/genai-phase2-templates.md")
print(f"• Issues GitHub: https://github.com/jsboige/CoursIA/issues")
print(f"• Logs détaillés: Activez debug_level = 'DEBUG'")

In [ ]:
# Export des résultats et génération de rapport
if export_results:
    print("\n📄 EXPORT DES RÉSULTATS")
    print("=" * 30)
    
    # Compilation des résultats
    export_data = {
        "configuration_test": {
            "timestamp": datetime.now().isoformat(),
            "notebook_version": "1.0.0",
            "test_parameters": {
                "test_openrouter": test_openrouter,
                "test_openai_direct": test_openai_direct,
                "run_benchmarks": run_benchmarks,
                "benchmark_iterations": benchmark_iterations,
                "benchmark_timeout": benchmark_timeout
            }
        },
        "api_configuration": {api: {k: v for k, v in config.items() if k != 'key_value'} for api, config in api_config.items()},
        "connectivity_results": connectivity_results,
        "benchmark_results": benchmark_results if 'benchmark_results' in locals() else [],
        "rate_limit_results": rate_limit_results if 'rate_limit_results' in locals() else [],
        "diagnostics": diag_results,
        "summary": {
            "total_apis_tested": len(connectivity_results),
            "successful_connections": sum(1 for r in connectivity_results if r['success']),
            "average_response_time": sum(r['response_time'] for r in connectivity_results if r['success']) / len([r for r in connectivity_results if r['success']]) if connectivity_results else 0,
            "configuration_status": "healthy" if all(r['success'] for r in connectivity_results) else "issues_detected"
        }
    }
    
    # Sauvegarde JSON
    output_dir = GENAI_ROOT / 'outputs' / 'api_configuration'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    json_file = output_dir / f"api_config_test_{timestamp}.json"
    
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)
    
    print(f"💾 Résultats exportés: {json_file}")
    print(f"📊 Données incluses: {len(export_data)} sections")
    
    # Génération rapport HTML simple si demandé
    if generate_report:
        html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Rapport Configuration APIs CoursIA</title>
    <meta charset="utf-8">
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        .header {{ background: #f0f8ff; padding: 20px; border-radius: 8px; }}
        .section {{ margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 5px; }}
        .success {{ color: #28a745; }}
        .error {{ color: #dc3545; }}
        .warning {{ color: #ffc107; }}
        table {{ width: 100%; border-collapse: collapse; margin: 10px 0; }}
        th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        th {{ background-color: #f2f2f2; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>🔗 Rapport Configuration APIs CoursIA</h1>
        <p>Généré le: {datetime.now().strftime('%d/%m/%Y à %H:%M:%S')}</p>
    </div>
    
    <div class="section">
        <h2>📊 Résumé</h2>
        <ul>
            <li>APIs testées: {export_data['summary']['total_apis_tested']}</li>
            <li>Connexions réussies: {export_data['summary']['successful_connections']}</li>
            <li>Temps de réponse moyen: {export_data['summary']['average_response_time']:.3f}s</li>
            <li>Status global: {export_data['summary']['configuration_status']}</li>
        </ul>
    </div>
    
    <div class="section">
        <h2>🌐 Connectivité</h2>
        <table>
            <tr><th>API</th><th>Status</th><th>Temps (s)</th><th>Modèles</th><th>Erreur</th></tr>
        """
        
        for result in connectivity_results:
            status_class = "success" if result['success'] else "error"
            status_text = "✅ Succès" if result['success'] else "❌ Échec"
            response_time = f"{result['response_time']:.3f}" if result.get('response_time') else "N/A"
            models_count = result.get('models_count', 0)
            error = result.get('error', 'Aucune') if not result['success'] else 'Aucune'
            
            html_content += f"""
            <tr>
                <td>{result['name']}</td>
                <td class="{status_class}">{status_text}</td>
                <td>{response_time}</td>
                <td>{models_count}</td>
                <td>{error}</td>
            </tr>
            """
        
        html_content += """
        </table>
    </div>
    
    <div class="section">
        <h2>🔧 Diagnostic</h2>
        <h3>Problèmes détectés:</h3>
        <ul>
        """
        
        if diag_results['common_issues']:
            for issue in diag_results['common_issues']:
                html_content += f"<li class='error'>❌ {issue['issue']}<br><small>💡 {issue['solution']}</small></li>"
        else:
            html_content += "<li class='success'>✅ Aucun problème détecté</li>"
        
        html_content += """
        </ul>
        <h3>Recommandations:</h3>
        <ul>
        """
        
        for rec in diag_results['recommendations']:
            html_content += f"<li>{rec}</li>"
        
        html_content += """
        </ul>
    </div>
    
    <div class="section">
        <h2>📝 Fichiers générés</h2>
        <ul>
            <li>Données JSON: api_config_test_{timestamp}.json</li>
            <li>Rapport HTML: api_config_report_{timestamp}.html</li>
        </ul>
    </div>
</body>
</html>
        """
        
        html_file = output_dir / f"api_config_report_{timestamp}.html"
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print(f"📄 Rapport HTML généré: {html_file}")
        print(f"💡 Ouvrez le fichier HTML dans votre navigateur")
else:
    print(f"\n⏭️  Export désactivé (export_results = False)")

## 🎯 Résumé et Prochaines Étapes

### ✅ Configuration Validée

- [ ] **APIs configurées** : Clés et endpoints vérifiés
- [ ] **Connectivité testée** : Tous les endpoints fonctionnels
- [ ] **Performance mesurée** : Benchmarks et temps de réponse
- [ ] **Rate limits analysés** : Limites et recommandations
- [ ] **Coûts estimés** : Projections d'usage mensuel
- [ ] **Diagnostic complet** : Problèmes détectés et solutions

### 🚀 Utilisation des APIs

Maintenant que vos APIs sont configurées, vous pouvez utiliser :

1. **Notebooks Foundation** :
   - `01-1-OpenAI-DALL-E-3.ipynb` - Génération d'images
   - `01-2-GPT-5-Image-Generation.ipynb` - Analyse multimodale

2. **Notebooks Advanced** :
   - `02-1-Qwen-Image-Edit-2509.ipynb` - Édition d'images
   - `02-2-FLUX-1-Advanced-Generation.ipynb` - Génération avancée

3. **Notebooks Applications** :
   - `04-1-Educational-Content-Generation.ipynb` - Contenu pédagogique
   - `04-2-Creative-Workflows.ipynb` - Workflows créatifs

### 💡 Bonnes Pratiques

**✅ Configuration:**
- Gardez vos clés API sécurisées dans `.env`
- Testez régulièrement la connectivité
- Surveillez vos quotas et coûts
- Implémentez des retries pour la résilience

**❌ Évitez:**
- Partager vos clés API dans le code
- Ignorer les rate limits
- Utiliser des timeouts trop courts
- Oublier de monitorer les coûts

### 🔗 Ressources

- **Documentation OpenRouter** : [openrouter.ai/docs](https://openrouter.ai/docs)
- **API OpenAI** : [platform.openai.com/docs](https://platform.openai.com/docs)
- **Troubleshooting** : `docs/genai-troubleshooting-guide.md`
- **Templates** : `docs/genai-phase2-templates.md`

### 📊 Fichiers Générés

Ce notebook génère automatiquement :
- **Rapport JSON** : Données complètes de configuration
- **Rapport HTML** : Visualisation des résultats
- **Logs détaillés** : Pour le debugging

---

**🎉 Configuration terminée !** Vos APIs sont prêtes pour l'utilisation dans l'écosystème CoursIA GenAI.